# Building model

# Set up

In [1]:
import cobra, libsbml
import pandas as pd
from cobrafunctions.buildingediting import *
model = cobra.io.read_sbml_model("Models/PlantCoreMetabolism_v1_2_2.xml")

cobra\__init__.py:33 FutureWarning: Python 2 is reaching end of life (see https://www.python.org/dev/peps/pep-0373/) and many cobra dependencies have already dropped support. At the moment it *should* still work but we will no longer actively maintain Python 2 support.
No handlers could be found for logger "cobra.io.sbml"


In [2]:
cobra.io.validate_sbml_model("Models/PlantCoreMetabolism_v1_2_2.xml")

(<Model TomCoreOxford at 0x10ce6cf8>,
 {'COBRA_CHECK': [],
  'COBRA_ERROR': ['No objective coefficients in model. Unclear what should be optimized'],
  'COBRA_FATAL': [],
  'COBRA_WARNING': ['Model does not contain SBML fbc package information.',
   "SBML package 'layout' not supported by cobrapy, information is not parsed",
   "SBML package 'render' not supported by cobrapy, information is not parsed",
   'Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_UREA_m "UREA">',
   'Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_UREA_m "UREA">',
   'Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_VAL_c "VAL">',
   'Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_VAL_c "VAL">',
   'Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_GLYCEROL_3P_c "GLYCEROL-3P">',
   'Use of CHARGE 

### Add charge to model

In [3]:
charges = pd.read_csv("Inputs/charges.csv")
charges.set_index("Metabolite")
for index, row in charges.iterrows():
    model.metabolites.get_by_id(row[1]).charge = row[0]

# Changes to Both Cells

### K+

In [4]:
for metabolite in model.metabolites:
    if "KI_" in metabolite.id:
        metabolite.id = metabolite.id[:1] + metabolite.id[-2:]
        metabolite.name = metabolite.id
model.reactions.K_rev_vc.name = "K_cv"
model.reactions.K_rev_vc.id = "K_cv"
model.metabolites.K_e.charge = 1

### Cl-

In [5]:
addmetabolite(model, "Cl_e", "e", multi = False)
addmetabolite(model, "Cl_c", "c", multi = False)
addmetabolite(model, "Cl_v", "v", multi = False)
model.metabolites.Cl_e.notes = {'INCHI': ['InChI=1S/ClH/h1H/p-1'], 'SMILES': ['Cl-']}
model.metabolites.Cl_e.charge = -1
model.metabolites.Cl_c.notes = model.metabolites.Cl_e.notes.copy()
model.metabolites.Cl_c.charge = -1
model.metabolites.Cl_v.notes = model.metabolites.Cl_e.notes.copy()
model.metabolites.Cl_v.charge = -1

addreaction(model, "Cl_tx", multi = False)
model.reactions.Cl_tx.add_metabolites({
    "Cl_e": 1
})
model.reactions.Cl_tx.lower_bound = -1000

### CIT

In [6]:
for metabolite in model.metabolites:
    if "CIT_" in metabolite.id and "aCIT" not in metabolite.id:
        metabolite.charge = -3

In [7]:
model.metabolites.aCIT_v.charge = -2

### MAL

In [8]:
for metabolite in model.metabolites:
    if "MAL_" in metabolite.id and "aMAL" not in metabolite.id:
        metabolite.charge = -2

In [9]:
model.metabolites.aMAL_v.charge = -1

In [10]:
addmetabolite(model, "MAL_e", "e", multi = False)
addmetabolite(model, "aMAL_e", "e", multi = False)

Metabolite identifier,aMAL_e
Name,aMAL_e
Memory address,0x013cff898
Formula,None
Compartment,e
In 0 reaction(s),


In [11]:
model.metabolites.MAL_e.notes = model.metabolites.MAL_v.notes.copy()
model.metabolites.aMAL_e.notes = model.metabolites.aMAL_v.notes.copy()
model.metabolites.MAL_e.charge = model.metabolites.MAL_v.charge
model.metabolites.aMAL_e.charge = model.metabolites.aMAL_v.charge

In [12]:
addreaction(model, "MAL_tx", multi = False)
model.reactions.MAL_tx.add_metabolites({
    "MAL_e": 0.7,
    "aMAL_e": 0.3,
})
model.reactions.MAL_tx.lower_bound = -1000

### MAL Anion Export Channel (Cl/Mal)

In [13]:
addreaction(model, "MAL_ce", multi = False)
model.reactions.MAL_ce.name = "MAL R/S-Type Anion Channel"
model.reactions.MAL_ce.add_metabolites({
    "MAL_c": -1,
    "MAL_e": 0.7,
    "aMAL_e": 0.3,
    "PROTON_e": -0.3
})

### NADPHox

In [14]:
#change names to make consistent with the rest of the model
for reaction in model.reactions:
    if "NADPHox" in reaction.id:
        reaction.name = reaction.id[:7] + "_" + reaction.id[7:]
        reaction.id = reaction.id[:7] + "_" + reaction.id[7:]

## Only active in germinating seeds

Arabidopsis Peroxisomal Citrate Synthase Is Required for Fatty Acid Respiration and Seed Germination
Itsara Pracharoenwattana, Johanna E. Cornah, Steven M. Smith
The Plant Cell Jul 2005, 17 (7) 2037-2048; DOI: 10.1105/tpc.105.031856

Coordinate expression of transcriptionally regulated isocitrate lyase and malate synthase genes in Brassica napus L.
L Comai, R A Dietrich, D J Maslyar, C S Baden, J J Harada
The Plant Cell Mar 1989, 1 (3) 293-300; DOI: 10.1105/tpc.1.3.293

In [15]:
model = setflux(model, "MALSYN_RXN_x", 0, 0, multi = False)

In [16]:
model = setflux(model, "CITSYN_RXN_x", 0, 0, multi = False)

## Split Model into Mesophyll (me) and Guard Cell (gc)

In [17]:
splitmodel(model, ["me", "gc"])

# Adding GC-specific transporters 

In [18]:
model.reactions.PROTON_ATPase_c_gc

Reaction identifier,PROTON_ATPase_c_gc
Name,PROTON_ATPase_c_gc
Memory address,0x015df3c50
Stoichiometry,0.65 ATP_c_gc + 0.45 PROTON_c_gc + WATER_c_gc + 0.35 aATP_c_gc --> 0.5 ADP_c_gc + PROTON_e_gc + 0.7 Pi_c_gc + 0.5 aADP_c_gc + 0.3 aPi_c_gc 0.65 ATP_gc + 0.45 PROTON_gc + WATER_gc + 0.35 aATP[c]_gc --> 0.5 ADP_gc + PROTON_gc + 0.7 Pi[c]_gc + 0.5 aADP[c]_gc + 0.3 aPi[c]_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Inward-rectifying K+ Channel

#### Remove dependence for protons

In [19]:
model.reactions.K_ec_gc.name = "Apoplastic Inward-Rectifying K+ Channel"
model.reactions.K_ec_gc.add_metabolites({
    "PROTON_e_gc": 1,
    "PROTON_c_gc": -1
})

In [20]:
model.reactions.K_ec_gc

Reaction identifier,K_ec_gc
Name,Apoplastic Inward-Rectifying K+ Channel
Memory address,0x015d33588
Stoichiometry,K_e_gc --> K_c_gc K_e_gc --> K_c_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Outward-rectifying K+ Channel

In [21]:
addreaction(model, "K_ce_gc", multi = False)
model.reactions.K_ce_gc.name = "Apoplastic Outward-Rectifying K+ Channel"
model.reactions.K_ce_gc.add_metabolites({
    "K_c_gc": -1,
    "K_e_gc": 1
})

### H+-coupled K+ Symport

In [22]:
addreaction(model, "K_PROTON_ec_gc", multi = False)
model.reactions.K_PROTON_ec_gc.name = "H+-Coupled K+ Symport"
model.reactions.K_PROTON_ec_gc.add_metabolites({
    "K_e_gc": -1,
    "K_c_gc": 1,
    "PROTON_e_gc": -1,
    "PROTON_c_gc": 1
})
model.reactions.K_PROTON_ec_gc

Reaction identifier,K_PROTON_ec_gc
Name,H+-Coupled K+ Symport
Memory address,0x010778160
Stoichiometry,K_e_gc + PROTON_e_gc --> K_c_gc + PROTON_c_gc K_e_gc + PROTON_gc --> K_c_gc + PROTON_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Cl- Import Channel

In [23]:
addreaction(model, "Cl_ec_gc", multi = False)
model.reactions.Cl_ec_gc.name = "Cl Apoplastic Import Channel"
model.reactions.Cl_ec_gc.add_metabolites({
    "Cl_e_gc": -1,
    "Cl_c_gc": 1
})

### H+-Coupled Cl- Symport

In [24]:
addreaction(model, "Cl_PROTON_ec_gc", multi = False)
model.reactions.Cl_PROTON_ec_gc.name = "H+-Coupled Cl- Symport"
model.reactions.Cl_PROTON_ec_gc.add_metabolites({
    "Cl_e_gc": -2,
    "Cl_c_gc": 2,
    "PROTON_e_gc": -1,
    "PROTON_c_gc": 1
})

### Cl- Anion Export Channel (Cl/Mal)

In [25]:
addreaction(model, "Cl_ce_gc", multi =False)
model.reactions.Cl_ce_gc.name = "Cl- R/S-Type Anion Channel"
model.reactions.Cl_ce_gc.add_metabolites({
    "Cl_c_gc": -1,
    "Cl_e_gc": 1
})

### ATPase Malate Importer

In [26]:
addreaction(model, "MAL_ATPASE_ec_gc", multi = False)
model.reactions.MAL_ATPASE_ec_gc.add_metabolites({
    "MAL_e_gc": -0.7,
    "aMAL_e_gc": -0.3,
    "MAL_c_gc": 1,
    "ATP_c_gc": -0.65,
    "aATP_c_gc": -0.35,
    "WATER_c_gc": -1,
    "ADP_c_gc": 1,
    "Pi_c_gc": 0.7,
    "aPi_c_gc": 0.3,
    "PROTON_c_gc": +0.85,
})

### Glucose Apoplastic Symport Channel

In [27]:
model.reactions.GLC_ec_gc.id = "GLC_PROTON_ec_gc"
model.reactions.GLC_PROTON_ec_gc.name = "H+-Coupled Glucose Symport"
model.reactions.GLC_PROTON_ec_gc

Reaction identifier,GLC_PROTON_ec_gc
Name,H+-Coupled Glucose Symport
Memory address,0x015b3fe10
Stoichiometry,GLC_e_gc + PROTON_e_gc --> GLC_c_gc + PROTON_c_gc GLC_gc + PROTON_gc --> GLC_gc + PROTON_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Glucose Exporter

In [28]:
addreaction(model, "GLC_ce_gc", multi = False)
model.reactions.GLC_ce_gc.name = "Glucose Apoplastic Exporter"
model.reactions.GLC_ce_gc.add_metabolites({
    "GLC_c_gc": -1,
    "GLC_e_gc": 1
})

### Sucrose Apoplastic Symport Channel

In [29]:
# Renaming Sucrose_tx
model.reactions.Sucrose_tx_gc.id = "SUCROSE_tx_gc"

In [30]:
model.reactions.Sucrose_ec_gc.id = "SUCROSE_PROTON_ec_gc"
model.reactions.SUCROSE_PROTON_ec_gc.name = "H+-Coupled Sucrose Symport"
model.reactions.SUCROSE_PROTON_ec_gc.upper_bound = 1000
model.reactions.SUCROSE_PROTON_ec_gc

Reaction identifier,SUCROSE_PROTON_ec_gc
Name,H+-Coupled Sucrose Symport
Memory address,0x016003588
Stoichiometry,PROTON_e_gc + SUCROSE_e_gc --> PROTON_c_gc + SUCROSE_c_gc PROTON_gc + SUCROSE_gc --> PROTON_gc + SUCROSE_gc
GPR,
Lower bound,0.0
Upper bound,1000


### Sucrose Exporter

In [31]:
addreaction(model, "SUCROSE_ce_gc", multi = False)
model.reactions.SUCROSE_ce_gc.name = "Sucrose Apoplastic Exporter"
model.reactions.SUCROSE_ce_gc.add_metabolites({
    "SUCROSE_c_gc": -1,
    "SUCROSE_e_gc": 1
})

### Fructose Apoplastic Symport Channel

In [32]:
addmetabolite(model, "FRU_e_gc", "e", multi = False)
model.metabolites.FRU_e_gc.notes = model.metabolites.FRU_c_gc.notes.copy()
model.metabolites.FRU_e_gc.charge = 0

In [33]:
addreaction(model, "FRU_PROTON_ec_gc", multi = False)
model.reactions.FRU_PROTON_ec_gc.name = "H+-Coupled Fructose Symport"
model.reactions.FRU_PROTON_ec_gc.add_metabolites({
    "FRU_e_gc": -1,
    "FRU_c_gc": 1,
    "PROTON_e_gc": -1,
    "PROTON_c_gc": 1
})

### Fructose Exporter

In [34]:
addreaction(model, "FRU_ce_gc", multi = False)
model.reactions.FRU_ce_gc.name = "Fructose Apoplastic Exporter"
model.reactions.FRU_ce_gc.add_metabolites({
    "FRU_c_gc": -1,
    "FRU_e_gc": 1
})

### Cell-Wall Invertase

In [35]:
addreaction(model, "cwINV_gc", multi = False)

Reaction identifier,cwINV_gc
Name,cwINV_gc
Memory address,0x01078cf28
Stoichiometry,--> -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [36]:
model.reactions.cwINV_gc.add_metabolites({
    "SUCROSE_e_gc": -1,
    "WATER_e_gc": -1,
    "FRU_e_gc": 1,
    "GLC_e_gc": 1
})

### K+ Import Channel

(TPK-Type and Fast-vacuolar)

In [37]:
model.reactions.K_cv_gc.name = "K+ Tonoplastic Import Channel"
model.reactions.K_cv_gc

Reaction identifier,K_cv_gc
Name,K+ Tonoplastic Import Channel
Memory address,0x015e59c50
Stoichiometry,K_c_gc + PROTON_v_gc --> K_v_gc + PROTON_c_gc K_c_gc + PROTON_gc --> K_v_gc + PROTON_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### NHX-Type K+/H+ Antiport

In [38]:
addreaction(model, "K_PROTON_cv_gc", multi = False)
model.reactions.K_PROTON_cv_gc.name = "NHX-Type K+/H+ Antiport"
model.reactions.K_PROTON_cv_gc.add_metabolites({
    "K_c_gc": -1,
    "K_v_gc": 1,
    "PROTON_v_gc": -1,
    "PROTON_c_gc": 1
})

### K+ Tonoplastic Export Channel

Slow Vacuolar Channel

In [39]:
addreaction(model, "K_vc_gc", multi = False)
model.reactions.K_vc_gc.name = "K+ Tonoplastic Export Slow Vacuolar Channel"
model.reactions.K_vc_gc.add_metabolites({
    "K_v_gc": -1,
    "K_c_gc": 1
})

### CLC-Type Cl-/H+ Antiport Vacoular Import Channel

In [40]:
addreaction(model, "Cl_PROTON_cv_gc", multi = False)
model.reactions.Cl_PROTON_cv_gc.name = "CLC-Type Cl-/H+ Antiport Vacoular Import Channel"
model.reactions.Cl_PROTON_cv_gc.add_metabolites({
    "Cl_v_gc": 1,
    "Cl_c_gc": -1,
    "PROTON_c_gc": 2,
    "PROTON_v_gc": -2,
})

### VCL Cl- Vacuolar Import Channel

In [41]:
addreaction(model, "Cl_cv_gc", multi = False)
model.reactions.Cl_cv_gc.name = "VCL Cl- Vacuolar Import Channel"
model.reactions.Cl_cv_gc.add_metabolites({
    "Cl_c_gc": -1,
    "Cl_v_gc": 1
})

### Cl- Vacuolar Export Channel

In [42]:
model.reactions.Cl_cv_gc.lower_bound = -1000

### MAL Import Channel (AMLT)

In [43]:
model.reactions.MAL_PROTON_vc_gc.id = "MAL_cv_gc"
model.reactions.MAL_cv_gc.name = "VMAL-type MAL Channel (Import)"
model.reactions.MAL_cv_gc

Reaction identifier,MAL_cv_gc
Name,VMAL-type MAL Channel (Import)
Memory address,0x015e73a90
Stoichiometry,MAL_c_gc + 0.3 PROTON_v_gc --> 0.7 MAL_v_gc + 0.3 aMAL_v_gc MAL_gc + 0.3 PROTON_gc --> 0.7 MAL_gc + 0.3 aMAL[v]_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### MAL Export Channel (AMLT)

In [44]:
addreaction(model, "MAL_vc_gc", multi = False)
model.reactions.MAL_vc_gc.name = "VMAL-type MAL Channel (Export)"
model.reactions.MAL_vc_gc.add_metabolites({
    "MAL_v_gc" : -0.7,
    "aMAL_v_gc": -0.3,
    "MAL_c_gc" : 1,
    "PROTON_c_gc": 0.3,
})

### MAL Export Symporter

Already in Model

In [45]:
model.reactions.MAL_PROTON_rev_vc_gc.name = "MAL Export Symporter"
model.reactions.MAL_PROTON_rev_vc_gc

Reaction identifier,MAL_PROTON_rev_vc_gc
Name,MAL Export Symporter
Memory address,0x01602f400
Stoichiometry,0.7 MAL_v_gc + 1.7 PROTON_v_gc + 0.3 aMAL_v_gc --> MAL_c_gc + 2.0 PROTON_c_gc 0.7 MAL_gc + 1.7 PROTON_gc + 0.3 aMAL[v]_gc --> MAL_gc + 2.0 PROTON_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Fructose Import Antiporter

Previously in the model

In [46]:
model.reactions.FRU_PROTON_rev_vc_gc.id = "FRU_PROTON_rev_cv_gc"
model.reactions.FRU_PROTON_rev_cv_gc.name = "Fructose Tonoplastic Import Antiporter"
model.reactions.FRU_PROTON_rev_cv_gc

Reaction identifier,FRU_PROTON_rev_cv_gc
Name,Fructose Tonoplastic Import Antiporter
Memory address,0x0158711d0
Stoichiometry,FRU_c_gc + PROTON_v_gc --> FRU_v_gc + PROTON_c_gc FRU[c]_gc + PROTON_gc --> FRU[v]_gc + PROTON_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Fructose Import Symporter

In [47]:
addreaction(model, "FRU_PROTON_cv_gc", multi = False)
model.reactions.FRU_PROTON_cv_gc.name = "Fructose Tonoplastic Import Symporter"
model.reactions.FRU_PROTON_cv_gc.add_metabolites({
    "PROTON_c_gc": -1,
    "PROTON_v_gc": 1,
    "FRU_c_gc": -1,
    "FRU_v_gc": 1
})

### Fructose Export

In [48]:
model.reactions.FRU_vc_gc

Reaction identifier,FRU_vc_gc
Name,FRU_vc_gc
Memory address,0x015d33eb8
Stoichiometry,FRU_v_gc --> FRU_c_gc FRU[v]_gc --> FRU[c]_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Glucose Tonoplastic Antiporter

In [49]:
model.reactions.GLC_PROTON_rev_vc_gc.id = "GLC_PROTON_rev_cv_gc"
model.reactions.GLC_PROTON_rev_cv_gc.name = "Glucose Tonoplastic Import Antiporter"
model.reactions.GLC_PROTON_rev_cv_gc

Reaction identifier,GLC_PROTON_rev_cv_gc
Name,Glucose Tonoplastic Import Antiporter
Memory address,0x015a927f0
Stoichiometry,GLC_c_gc + PROTON_v_gc --> GLC_v_gc + PROTON_c_gc GLC_gc + PROTON_gc --> GLC_gc + PROTON_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Glucose Tonoplastic Symporter

In [50]:
addreaction(model, "GLC_PROTON_cv_gc", multi = False)
model.reactions.GLC_PROTON_cv_gc.name = "Glucose Tonoplastic Import Symporter"
model.reactions.GLC_PROTON_cv_gc.add_metabolites({
    "PROTON_c_gc": -1,
    "PROTON_v_gc": 1,
    "GLC_c_gc": -1,
    "GLC_v_gc": 1
})

### Glucose Export Channel

In [51]:
model.reactions.GLC_vc_gc

Reaction identifier,GLC_vc_gc
Name,GLC_vc_gc
Memory address,0x015e59f60
Stoichiometry,GLC_v_gc --> GLC_c_gc GLC_gc --> GLC_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Sucrose Tonoplastic Importer

In [52]:
addreaction(model, "SUCROSE_cv_gc", multi = False)
model.reactions.SUCROSE_cv_gc.name = "Sucrose Free Tonoplastic Import"
model.reactions.SUCROSE_cv_gc.add_metabolites({
    "SUCROSE_c_gc": -1,
    "SUCROSE_v_gc": 1,
})

### Sucrose Tonoplastic Import Antiporter

Already in the model

In [53]:
model.reactions.SUCROSE_PROTON_vc_gc.id = "SUCROSE_PROTON_cv_gc"
model.reactions.SUCROSE_PROTON_cv_gc.name = "Sucrose Tonoplastic Import Antiporter"
model.reactions.SUCROSE_PROTON_cv_gc

Reaction identifier,SUCROSE_PROTON_cv_gc
Name,Sucrose Tonoplastic Import Antiporter
Memory address,0x0157711d0
Stoichiometry,PROTON_v_gc + SUCROSE_c_gc --> PROTON_c_gc + SUCROSE_v_gc PROTON_gc + SUCROSE_gc --> PROTON_gc + SUCROSE_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Sucrose Tonoplastic Export Symporter

In [54]:
model.reactions.SUCROSE_PROTON_rev_vc_gc.id = "SUCROSE_PROTON_vc_gc"
model.reactions.SUCROSE_PROTON_vc_gc.name = "Sucrose Tonoplastic Export Symporter"

### Tonoplastic PPase

In [55]:
model.reactions.PROTON_PPi_rev_vc_gc

Reaction identifier,PROTON_PPi_rev_vc_gc
Name,PROTON_PPi_rev_vc_gc
Memory address,0x015f69ef0
Stoichiometry,0.65 PPI_c_gc + 0.25 PROTON_c_gc + WATER_c_gc + 0.35 aPPI_c_gc --> PROTON_v_gc + 1.4 Pi_c_gc + 0.6 aPi_c_gc 0.65 PPI_gc + 0.25 PROTON_gc + WATER_gc + 0.35 aPPI[c]_gc --> PROTON_gc + 1.4 Pi[c]_gc + 0.6 aPi[c]_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


### Tonoplastic H+ ATPase

In [56]:
model.reactions.PROTONATP_rev_vc_gc

Reaction identifier,PROTONATP_rev_vc_gc
Name,PROTONATP_rev_vc_gc
Memory address,0x015634d68
Stoichiometry,0.65 ATP_c_gc + 1.45 PROTON_c_gc + WATER_c_gc + 0.35 aATP_c_gc --> 0.5 ADP_c_gc + 2.0 PROTON_v_gc + 0.7 Pi_c_gc + 0.5 aADP_c_gc + 0.3 aPi_c_gc 0.65 ATP_gc + 1.45 PROTON_gc + WATER_gc + 0.35 aATP[c]_gc --> 0.5 ADP_gc + 2.0 PROTON_gc + 0.7 Pi[c]_gc + 0.5 aADP[c]_gc + 0.3 aPi[c]_gc
GPR,
Lower bound,0.0
Upper bound,1000.0


In [57]:
addreaction(model, "PROTON_ec_gc", multi = False)
model.reactions.PROTON_ec_gc.name = "Plasma membrane proton leakage"
model.reactions.PROTON_ec_gc.add_metabolites({
    "PROTON_e_gc": -1,
    "PROTON_c_gc": 1,
})

In [58]:
addreaction(model, "PROTON_vc_gc", multi = False)
model.reactions.PROTON_vc_gc.name = "Tonoplast membrane proton leakage"
model.reactions.PROTON_vc_gc.add_metabolites({
    "PROTON_v_gc": -1,
    "PROTON_c_gc": 1,
})

## Constrain Channels in Guard Cell

In [59]:
for channel in ["Cl_ec","K_cv","FRU_PROTON_cv","GLC_PROTON_cv","SUCROSE_cv"]:
    model = setflux(model, channel + "_gc", 0,0, multi = False)

# Adding apoplast and exchanges

### Removing tx_me

In [60]:
tx_me = []
for reaction in model.reactions:
    if "_tx_me" in reaction.id:
        tx_me.append(reaction)
keep = ["Photon_tx_me", "Phloem_output_tx_me", "NADPHox_c_tx_me", "NADPHox_m_tx_me", "NADPHox_p_tx_me", "O2_tx_me", "CO2_tx_me", "ATPase_tx_me"]
for reaction in keep:
    tx_me.remove(model.reactions.get_by_id(reaction))
model.remove_reactions(tx_me)

### Adding tx to apo and adding exchanges

In [61]:
ea_reactions = []
addmetabolite(model, "FRU_e_me", "e", multi = False)
for metabolite in model.metabolites:
    if "_e_gc" in metabolite.id:
        ea_reactions.append(metabolite)
remove = ["Photon", "OXYGEN_MOLECULE", "CARBON_DIOXIDE"]
for name in remove:
    ea_reactions.remove(model.metabolites.get_by_id(name+"_e_gc"))
for metabolite in ea_reactions:
    addmetabolite(model, metabolite.id[:-4]+"a", "a", multi = False)
    model.metabolites.get_by_id(metabolite.id[:-4]+"a").charge = metabolite.charge
    model.metabolites.get_by_id(metabolite.id[:-4]+"a").notes = metabolite.notes
    addreaction(model, metabolite.id[:-4]+"a_tx", multi = False)
    model.reactions.get_by_id(metabolite.id[:-4]+"a_tx").add_metabolites({
        metabolite.id[:-4]+"a": 1
    })
    model.reactions.get_by_id(metabolite.id[:-4]+"a_tx").lower_bound = -1000
    addreaction(model, metabolite.id[:-4]+"ae_gc", multi = False)
    model.reactions.get_by_id(metabolite.id[:-4]+"ae_gc").add_metabolites({
        metabolite:1,
        metabolite.id[:-4]+"a": -1
    })
    model.reactions.get_by_id(metabolite.id[:-4]+"ae_gc").lower_bound = -1000
    addreaction(model, metabolite.id[:-4]+"ae_me", multi = False)
    model.reactions.get_by_id(metabolite.id[:-4]+"ae_me").add_metabolites({
        metabolite.id[:-4]+"e_me":1,
        metabolite.id[:-4]+"a": -1
    })
    model.reactions.get_by_id(metabolite.id[:-4]+"ae_me").lower_bound = -1000

### Removing tx_gc

In [62]:
tx_gc = []
for reaction in model.reactions:
    if "_tx_gc" in reaction.id:
        tx_gc.append(reaction)
keep = ["Photon_tx_gc", "NADPHox_c_tx_gc", "NADPHox_m_tx_gc", "NADPHox_p_tx_gc", "O2_tx_gc", "CO2_tx_gc", "ATPase_tx_gc"]
for reaction in keep:
    tx_gc.remove(model.reactions.get_by_id(reaction))

model.remove_reactions(tx_gc)

In [63]:
for reaction in model.reactions:
    if "_tx" in reaction.id:
        print reaction.id

Photon_tx_me
Photon_tx_gc
NADPHox_m_tx_me
NADPHox_m_tx_gc
CO2_tx_me
CO2_tx_gc
O2_tx_me
O2_tx_gc
Phloem_output_tx_me
NADPHox_c_tx_me
NADPHox_c_tx_gc
ATPase_tx_me
ATPase_tx_gc
NADPHox_p_tx_me
NADPHox_p_tx_gc
NITRATE_a_tx
SUCROSE_a_tx
WATER_a_tx
MGII_a_tx
GLC_a_tx
PROTON_a_tx
CAII_a_tx
Pi_a_tx
SULFATE_a_tx
K_a_tx
AMMONIUM_a_tx
Cl_a_tx
MAL_a_tx
aMAL_a_tx
FRU_a_tx


In [64]:
addreaction(model, "Sucrose_ce_me", multi = False)
model.reactions.Sucrose_ce_me.add_metabolites({
    "SUCROSE_c_me": -1,
    "SUCROSE_e_me": 1
})

# Quadruple model into 4 phases

In [65]:
splitmodel(model, range(1,5))

# Constrain free exchange and starch breakdown

In [66]:
for reaction in ["GLC_a_tx","MAL_a_tx","SUCROSE_a_tx","FRU_a_tx","PROTON_a_tx","RXN_1826_p_me","RXN_1826_p_gc","MALTODEXGLUCOSID_RXN_p_me","MALTODEXGLUCOSID_RXN_p_gc"]:
    model = setflux(model, reaction, 0,0, multi = True)

# Add Linker Reactions With Pseudometabolites (Osmolarity and Charge)

In [67]:
compartments = {
    "c": 0.1,
    "v": 0.9,
    "p": 0,
    "a": 1
    }

cells = ["gc", "me",]
phasetimes = [6.0,0.5,11.5,6.0]

In [68]:
addlinkers(model, "Inputs/osmolytes.csv", compartments, cells, phasetimes)

# Add Maintenance Reactions

In [69]:
numberofmodels = checknumberofmodels(model)
for cell in ["gc", "me"]:
    addmetabolite(model, "maintenance_ratio_constraint_" + cell, "pseudo")
    addmetabolite(model, "maintenance_phase_constraint_" + cell, "pseudo")
    addreaction(model, "maintenance_phase_overall_" + cell, multi = "")
    for i in range(1,numberofmodels+1):
        for x in ["c", "m", "p"]:
            reaction = model.reactions.get_by_id("NADPHox_" + x + "_tx_" + cell + "_" + str(i))
            reaction.add_metabolites({
                "maintenance_ratio_constraint_" + cell + "_" + str(i): -3
            })
        reaction = model.reactions.get_by_id("ATPase_tx_" + cell + "_" + str(i))
        reaction.add_metabolites({
            "maintenance_ratio_constraint_" + cell + "_" + str(i): 1, 
            "maintenance_phase_constraint_" + cell + "_" + str(i): 1
        })
        model.reactions.get_by_id("maintenance_phase_overall_" + cell).add_metabolites({
            "maintenance_phase_constraint_" + cell + "_" + str(i): -1
        })

# Add Phloem_tx Overall

In [70]:
numberofmodels = checknumberofmodels(model)
day = [2,3]
night = [1,4]
addmetabolite(model,"pseudoPhloem_me", "pseudo")
addmetabolite(model, "pseudoPhloem_day_me", "pseudo", multi = "")
addmetabolite(model, "pseudoPhloem_night_me", "pseudo", multi = "")
addreaction(model, "Phloem_constraint_day", multi ="")
addreaction(model, "Phloem_constraint_night", multi ="")
addreaction(model, "Phloem_tx_overall", multi = "")
for i in range(1,numberofmodels+1):
    lengthofphase = 1/(-model.reactions.get_by_id("SUCROSE_v_gc_Linker_" + str(i)).get_coefficient("SUCROSE_v_gc_" + str(i)))
    model.reactions.get_by_id("Phloem_output_tx_me_" + str(i)).add_metabolites({
        "pseudoPhloem_me_" +str(i): 1
    })
    if i in day:
        model.reactions.get_by_id("Phloem_output_tx_me_" + str(i)).add_metabolites({
            "pseudoPhloem_day_me": 1*lengthofphase
        })
        model.reactions.Phloem_constraint_day.add_metabolites({
            "pseudoPhloem_me_" +str(i): -1
        })
    elif i in night:
        model.reactions.get_by_id("Phloem_output_tx_me_" + str(i)).add_metabolites({
            "pseudoPhloem_night_me": 1*lengthofphase
        })
        model.reactions.Phloem_constraint_night.add_metabolites({
                "pseudoPhloem_me_" +str(i): -1
            })
    else:
        raise UserError("Make sure all phases are either assigned to day or night")
model.reactions.Phloem_constraint_day.add_metabolites({
    
})
model.reactions.Phloem_tx_overall.add_metabolites({
    "pseudoPhloem_day_me": -3,
    "pseudoPhloem_night_me": -1,
})

# Constrain Photons

In [71]:
model = setflux(model, "Photon_tx_me_1", 0, 0, multi = False)
model = setflux(model, "Photon_tx_me_2", 0, 9715, multi = False)
model = setflux(model, "Photon_tx_me_3", 0, 9715, multi = False)
model = setflux(model, "Photon_tx_me_4", 0, 0, multi = False)

model = setflux(model, "Photon_tx_gc_1", 0, 0, multi = False)
model = setflux(model, "Photon_tx_gc_2", 0, 0.462, multi = False)
model = setflux(model, "Photon_tx_gc_3", 0, 0.462, multi = False)
model = setflux(model, "Photon_tx_gc_4", 0, 0, multi = False)

# Constrain CO2

In [72]:
openco2flux = ((42.2*11)/(0.00415+11**0.5))+0.028
closedco2flux = ((42.2*6)/(0.00415+6**0.5))+0.028

model = setflux(model, "CO2_tx_me_1", -closedco2flux, closedco2flux, multi = False)
model = setflux(model, "CO2_tx_me_2", -openco2flux, openco2flux, multi = False)
model = setflux(model, "CO2_tx_me_3", -openco2flux, openco2flux, multi = False)
model = setflux(model, "CO2_tx_me_4", -closedco2flux, closedco2flux, multi = False)

model = setflux(model, "CO2_tx_gc_1", -1000, 1000, multi = False)
model = setflux(model, "CO2_tx_gc_2", -1000, 1000, multi = False)
model = setflux(model, "CO2_tx_gc_3", -1000, 1000, multi = False)
model = setflux(model, "CO2_tx_gc_4", -1000, 1000, multi = False)

# Export Model

In [73]:
cobra.io.write_sbml_model(model, "Models/4_stage_GC.xml")
cobra.io.write_sbml_model(model, "Inputs/4_stage_GC.xml")